# Introduction

This project used a combination of spaCy's Named Entity Recognition (NER) model and regular expressions to automate the extraction of client names and proposal dates from thousands of service proposals in an accounting firm's proposal archive. This significantly reduced the amount of manual effort required for data cleaning. The extracted information was then saved to an Excel file for future use.

In [ ]:
!pip install pdfminer.six

In [ ]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
# import spaCy for NLP
import spacy

# import os to get the local pdf files
import os

# import StringIO and pdfminer to extract text from pdf 
from io import StringIO
from pdfminer.high_level import extract_text_to_fp

# import regular expression to find pattern in text
import re

# import pandas to export the data into an excel file
import pandas as pd

In [ ]:
# a function to extract text from user-defined pages of a pdf file
def extract_text_from_pages(file, *args):
    try:
        # open pdf file
        with open(file, 'rb') as file:
          
            # extract text from pdf
            string_io = StringIO()
            extract_text_to_fp(file, string_io, page_numbers = [arg-1 for arg in args])
            extracted_text = string_io.getvalue()

            # clean text - replace multiple whitespaces characters with a single whitespace
            extracted_text = re.sub(r'\s+', ' ', extracted_text)
            
    except:
        # the message when pdfminer fails to extract text from pdf
        extracted_text = 'PDF Access Denied'
    
    return extracted_text

In [ ]:
# load spacy pre-trained model
nlp = spacy.load('en_core_web_lg')

# get the "Named Entity Recognition" pipeline
ner = nlp.get_pipe('ner')

# configure the pipeline to lower threshold to make it more sensitive to a potential organization name
ner.cfg['threshold'] = 0.01

In [ ]:
# directory path where the PDF files are stored
pdf_dir = 'C:/Users/directory/'

## 1. Extract Client Names From Proposals

In [ ]:
# a function to clean the organization names for better named entity recognition
def clean_organization_name(text):
    
    # clean the text
    text = text.replace(',LLC', ' LLC')
    text = text.replace(', LLC', ' LLC')
    text = text.replace(',LP', ' L.P.')
    text = text.replace(', LP', ' L.P.')
    text = text.replace(' LP', ' L.P.')
    text = text.replace(', L.P.', ' L.P.')
    text = text.replace(',L.P.', ' L.P.')
    text = text.replace(', Inc', ' Inc')
    cleaned_text = text.replace(',Inc', ' Inc')
    
    return cleaned_text

In [ ]:
# a function to run NER model on a text to extract entities labeled 'ORG'
def ner_org(text):
    
    # use spaCy nlp pipeline to analyze the text                
    doc = nlp(text)
                    
    # extract the named entities labeled 'ORG' and exclude the service provider's name
    organizations = set([ent.text for ent in doc.ents if ent.label_ == 'ORG' and 'provider_name' not in ent.text.lower()])

    return organizations

In [ ]:
# a combined function
def clean_and_ner_org(text):
    
    cleaned_text = clean_organization_name(text)
    organizations = ner_org(cleaned_text)
    
    return organizations

In [ ]:
# define the patterns we want to identify in the text

# 1: "Dear xxx: We are delighted to have this opportunity to propose on audit and tax services for XYZ(client name)"
pattern_1 = r'Dear(.*?)\('

# 2: In the cover page(first page) "XYZ(client name) Proposal to Provide Professional Services"
pattern_2 = r'.+(?=Proposal)'

In [ ]:
# create an empty list to store the data that need to be exported to excel
data_organization = []

# loop through all pdf files in a folder
for filename in os.listdir(pdf_dir):
    if filename.endswith('.pdf'):
        
        # extract the first 2 pages
        extracted_text = extract_text_from_pages(pdf_dir + filename, 1, 2)
        
        # check if we can extract text from the pdf file
        if extracted_text == 'PDF Access Denied':
            # put the message into the data list
            data_organization.append([filename, extracted_text, extracted_text])
        
        else:
            # search for the pattern_1 in the first 2 pages
            match_1 = re.search(pattern_1, extracted_text)
                
            if match_1:
                # get the matched text
                match_1_text = match_1.group(0)
                
                # clean the text and identify organizations
                organizations = clean_and_ner_org(match_1_text)

                # check if the model indentified any organizations from the matched text
                if len(organizations) != 0:
                    # put the pdf file name, identified organizations and matched text into the data list
                    for organization in organizations:
                        data_organization.append([filename, organization, match_1_text.replace('\n',' ')])
        
                else: 
                    # if the model failed to identify any organizations from the matched text
                    # get the text of the first page, which is the cover page of a proposal, and in most cases it contains the client name
                    extracted_text = extract_text_from_pages(pdf_dir + filename, 1)
                    
                    # search for the pattern_2 in the first page
                    match_2 = re.search(pattern_2, extracted_text)
                    
                    if match_2:
                        # get the text before "Proposal"
                        organization = match_2.group(0).strip()
                        data_organization.append([filename, organization, extracted_text.replace('\n',' ')])
        
                    else: 
                        # put 'Organization Not Found' message into the data list if failed to match pattern_2
                        data_organization.append([filename, 'Organization Not Found', extracted_text.replace('\n',' ')])
                        
            else:
                # get the text of the first page if we didn't find the matched pattern_1 from the first 2 pages
                extracted_text = extract_text_from_pages(pdf_dir + filename, 1)
            
                # search for the pattern_2 in the first page
                match_2 = re.search(pattern_2, extracted_text)
                
                if match_2:
                    # get the text before "Proposal"
                    organization = match_2.group(0).strip()
                    data_organization.append([filename, organization, extracted_text.replace('\n',' ')])
        
                else: 
                    # put 'Organization Not Found' message into the data list if failed to match pattern_2
                    data_organization.append([filename, 'Organization Not Found', extracted_text.replace('\n',' ')])

In [ ]:
# convert the data list to a pandas DataFrame
df_organization = pd.DataFrame(data_organization, columns=['Proposal File Name', 'Client Name', 'Snippet']) 

## 2. Extract Proposal Dates From Proposals

In [ ]:
# use regular expression to describe the proposal date pattern
date_pattern = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{1,2}[, ]*\d{4}|\b\d{1,2}[/-]\d{1,2}[/-]\d{2,4}\b|\b\d{4}[/-]\d{1,2}[/-]\d{1,2}\b'

In [ ]:
# create an empty list to store the data that need to be exported to excel
data_date = []

# loop through all pdf files in a folder
for filename in os.listdir(pdf_dir):
    if filename.endswith('.pdf'):
        
        # extract the first page
        extracted_text = extract_text_from_pages(pdf_dir + filename, 1)
        
        # check if we can extract text from the pdf file
        if extracted_text == 'PDF Access Denied':
            # put the message into the data list
            data_date.append([filename, extracted_text, extracted_text])
        
        else:
            # clean the text
            extracted_text = re.sub(r'\s+', ' ', extracted_text)

            # search for date pattern and put identified dates into the data list
            dates = re.findall(date_pattern, extracted_text)
            for date in dates:
                data_date.append([filename, date, extracted_text])

In [ ]:
# convert the data list to a pandas DataFrame
df_date = pd.DataFrame(data_date, columns=['Proposal File Name', 'Proposal Date', 'Snippet']) 

## 3. Export Data into Excel

In [ ]:
# export dataframes to an excel file
with pd.ExcelWriter('spaCy_NER.xlsx', engine='openpyxl') as writer:

    # create the first sheet
    writer.book.create_sheet('Client Name')
    df_organization.to_excel(writer, sheet_name = 'Client Name', index=True)

    # create the second sheet
    writer.book.create_sheet('Proposal Date')
    df_date.to_excel(writer, sheet_name = 'Proposal Date', index=True)